---

### Partie 1 : Les Questions de Recherche (À donner au début)

L'objectif est de les forcer à comprendre la **signature** de l'attaque qu'ils vont devoir détecter.

**Thème 1 : Injection SQL (SQLi)**

> *Question :* Lisez la documentation OWASP sur l'Injection SQL. Dans une attaque classique "in-band", quels caractères ou mots-clés un attaquant injecte-t-il souvent dans l'URL pour contourner une authentification (ex: se connecter sans mot de passe) ?
> *Ressource :* [OWASP SQL Injection]()
> *Réponse attendue :* L'apostrophe (`'`), le commentaire (`--` ou `#`), et l'opérateur logique `OR 1=1`.

**Thème 2 : Path Traversal (LFI)**

> *Question :* Une attaque par traversée de répertoire (Path Traversal) cherche à accéder à des fichiers hors du dossier web. Quelle séquence de caractères est typique de cette attaque ? Quel fichier système Linux contenant les utilisateurs est souvent ciblé ?
> *Ressource :* [PortSwigger Path Traversal]()
> *Réponse attendue :* La séquence `../` (dot-dot-slash) et le fichier `/etc/passwd`.

**Thème 3 : Déni de Service (Slowloris)**

> *Question :* Contrairement à une attaque volumétrique (inonder de trafic), comment fonctionne l'attaque Slowloris ? Que fait-elle aux connexions HTTP pour bloquer le serveur ?
> *Ressource :* [Cloudflare Slowloris]()
> *Réponse attendue :* Elle ouvre beaucoup de connexions mais ne les termine jamais (envoie des en-têtes très lentement), épuisant ainsi le "pool" de connexions du serveur.

---

In [11]:
# --- CELLULE 1 : IMPORT & CONFIGURATION ---
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Configuration pour Docker (Loki est accessible via son nom de service)
#LOKI_URL = "http://loki:3100/loki/api/v1/query_range"
LOKI_URL = "http://localhost:3100/loki/api/v1/query_range"

def query_loki(query, minutes=10):
    """
    Fonction utilitaire pour interroger Loki.
    Retourne une liste de résultats ou une liste vide.
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=minutes)
    
    params = {
        'query': query,
        'start': int(start_time.timestamp() * 1e9),
        'end': int(end_time.timestamp() * 1e9),
        'limit': 1000,
        'direction': 'BACKWARD'
    }
    
    try:
        response = requests.get(LOKI_URL, params=params)
        response.raise_for_status()
        data = response.json()
        return data['data']['result']
    except Exception as e:
        print(f"❌ Erreur API Loki : {e}")
        return []

print("✅ Configuration chargée. Prêt pour le Threat Hunting.")



✅ Configuration chargée. Prêt pour le Threat Hunting.


In [12]:
# --- CELLULE 2 : EXERCICE 1 - DÉTECTION SSH (RAPPEL) ---
# Objectif : Retrouver les IPs qui bruteforcent le serveur SSH.

# SOLUTION PROFESSEUR
print("--- 🕵️‍♂️ ANALYSE SSH ---")

# 1. On cherche le message "Failed password" dans le conteneur victim_server
query_ssh = '{container="victim_server"} |= "Failed password"'
logs = query_loki(query_ssh, minutes=5)

if logs:
    print(f"🔴 {len(logs)} logs suspects trouvés.")
    # Extraction simple pour l'affichage
    for log in logs[:3]:
        print(f"   Exemple: {log['values'][0][1]}")
else:
    print("🟢 Aucun bruteforce SSH détecté pour le moment.")

--- 🕵️‍♂️ ANALYSE SSH ---
🔴 1 logs suspects trouvés.
   Exemple: Failed password for root from 172.26.0.8 port 48054 ssh2


In [13]:
# --- CELLULE 3 : EXERCICE 2 - DÉTECTION INJECTION SQL (NOUVEAU) ---
# Consigne : L'attaquant tente d'injecter du SQL dans l'URL. 
# Indice : Cherchez les motifs '%27' (apostrophe encodée), 'OR', 'SELECT' dans l'URI.

print("\n--- 🕵️‍♂️ ANALYSE WEB : SQL INJECTION ---")

# SOLUTION PROFESSEUR
# On filtre les logs Nginx (JSON) où l'URI contient des motifs SQLi typiques
# Note : %27 est l'encodage URL pour l'apostrophe (')
query_sqli = '{container="web_server"} | json | request_uri =~ ".*(%27|OR|1%3D1|UNION|SELECT).*"'

logs = query_loki(query_sqli, minutes=5)

if logs:
    count = sum(len(entry['values']) for entry in logs)
    print(f"🚨 ALERTE CRITIQUE : {count} tentatives d'Injection SQL !")
    
    # Analyse de l'attaquant (IP Source)
    attackers = set()
    for stream in logs:
        # Comme on a parsé le JSON côté Loki, on peut récupérer les labels ou parser le log brut ici
        # Dans notre cas, Loki retourne le stream brut, on extrait l'IP du JSON dans 'stream' si dispo
        # ou on regarde le log brut.
        pass 
    print("   ↳ Signature détectée : ' OR 1=1 --")
else:
    print("🟢 Pas d'injection SQL détectée.")


--- 🕵️‍♂️ ANALYSE WEB : SQL INJECTION ---
🚨 ALERTE CRITIQUE : 500 tentatives d'Injection SQL !
   ↳ Signature détectée : ' OR 1=1 --


In [14]:
# --- CELLULE 4 : EXERCICE 3 - DÉTECTION PATH TRAVERSAL (NOUVEAU) ---
# Consigne : L'attaquant essaie de lire /etc/passwd.
# Indice : Cherchez la séquence ".." ou le nom du fichier sensible.

print("\n--- 🕵️‍♂️ ANALYSE WEB : PATH TRAVERSAL ---")

# SOLUTION PROFESSEUR
# On cherche l'accès à "etc/passwd" ou des ".."
query_traversal = '{container="web_server"} | json | request_uri =~ ".*etc/passwd.*"'

logs = query_loki(query_traversal, minutes=5)

if logs:
    print(f"⚠️ ALERTE : Tentative d'accès fichier système ({len(logs)} logs)")
    print("   ↳ Cible : /etc/passwd")
else:
    print("🟢 Fichiers systèmes en sécurité.")


--- 🕵️‍♂️ ANALYSE WEB : PATH TRAVERSAL ---
⚠️ ALERTE : Tentative d'accès fichier système (51 logs)
   ↳ Cible : /etc/passwd


In [15]:
# --- CELLULE 5 : DASHBOARD RÉCAPITULATIF (BONUS) ---
# Création d'un petit tableau de bord en ASCII pour le chef SOC.

def generate_report():
    print("\n" + "="*40)
    print("   RAPPORT SOC - TEMPS RÉEL")
    print("="*40)
    
    attacks = {
        "SSH Bruteforce": '{container="victim_server"} |= "Failed password"',
        "SQL Injection ": '{container="web_server"} | json | request_uri =~ ".*(%27|OR).*"',
        "Path Traversal": '{container="web_server"} | json | request_uri =~ ".*passwd.*"',
        "Nmap Scan     ": '{container="web_server"} | json | http_user_agent =~ ".*Nmap.*"'
    }
    
    for name, query in attacks.items():
        res = query_loki(query, minutes=2)
        count = sum(len(r['values']) for r in res) if res else 0
        status = "🔴 DANGER" if count > 0 else "🟢 OK"
        print(f"[{status}] {name} : {count} evts/2min")

generate_report()


   RAPPORT SOC - TEMPS RÉEL
[🔴 DANGER] SSH Bruteforce : 126 evts/2min
[🔴 DANGER] SQL Injection  : 200 evts/2min
[🔴 DANGER] Path Traversal : 200 evts/2min
[🔴 DANGER] Nmap Scan      : 20 evts/2min
